## Comparative Timings for the Bankruptcy Model

### The Standard Bellman Equation

The value of repaying one's debts, $v^R$, satisfies the 
Bellman equation

\begin{equation*}
	v^R (d,z, \eta, \kappa) = \max_{c, \, d'}
	\left[
	u(c) + \beta E_{z', \eta', \kappa' \mid z} \max 
	\left\{
	v^R (d', z', \eta', \kappa'), \, v^B (z', \eta')
	\right\}
	\right]
\end{equation*}

subject to 

$$c + d + \kappa \leq \bar{e} z \eta + q (z) d' . $$

Here $v^B$ is the value of declaring bankruptcy, which satisfies

\begin{equation*}
	v^B (z, \eta) = u(c) + 
	\beta E_{z', \eta', \kappa' \mid z} \max 
	\left\{ 
	v^R (0, z', \eta', \kappa'), \, v^E (z', \eta', \kappa')
	\right\}
\end{equation*}

subject to 

$$ c = \hat c := (1 - \gamma) \bar{e} z \eta . $$ 

Finally, $v^E$ is the value of defaulting on expense debt, which satisfies 

\begin{equation*}
	v^E (z, \eta, \kappa) = u (c ) + 
	\beta E_{z', \eta', \kappa' \mid z} \max 
	\left\{
	v^R (d', z', \eta', \kappa'), \, v^B (z', \eta')
	\right\}
\end{equation*}

subject to 

$$ 
    c = \hat c
   \quad \text{and} \quad
   d' = \hat d := (\kappa - \gamma \bar{e} z \eta) (1 + \bar{r}) 
   .
$$

### The Refactored Bellman Equation

After the refactoring, the modified Bellman equations become

\begin{equation*}
	g^{D} (z, d') = 
	E_{z', \eta', \kappa' \mid z} \max 
	\left\{
	\max_{c', \, d''} 
	\left[
	u (c') + \beta g^{D} (z', d'')
	\right], \,
	u (\tilde{c}) + \beta g^{E} (z')
	\right\}
\end{equation*}

subject to

$$
    c'  =  z' \eta' + q(z') d'' - d' - \kappa'
    \quad \text{and} \quad
    \tilde{c} = (1 - \gamma)  z' \eta'
$$

and

\begin{equation*}
	g^{E} (z) = 
	E_{z', \eta', \kappa' \mid z} \max 
	\left\{ 
	\max_{c', \, d''} 
	\left[
	u (c') + \beta g^{D} (z', d'')
	\right], \,
	u (\tilde{c}) + \beta g^{D} (z', \tilde{d})
	\right\}
\end{equation*}

subject to 

$$
    c' = z' \eta' + q(z') d'' - \kappa' 
    \quad \text{and} \quad
    \tilde{d} = (\kappa' - \gamma z' \eta') (1 + \bar{r}).
$$


### Further Setup

> The utility function: $u(c) = c^{1-\sigma} / (1 - \sigma)$.

> The transitory component of productivity: $(\eta_t) \overset{iid}{\sim} N(0, \delta_\epsilon^2)$.
 
> The expense shock: $(\kappa_t) \overset{iid}{\sim} U[\kappa_{min}, \kappa_{max}]$.
 
> The persistent component of productivity:
\begin{equation*}
     \log z_{t+1} = \rho \log z_t + \epsilon_{t+1}, \quad
     (\epsilon_t) \overset{iid}{\sim} N (0, \delta_\epsilon^2).
\end{equation*}


In [1]:
import numpy as np
import quantecon as qe
from numba import njit
import time

In [2]:
class ConsumerBankruptcy:
    
    def __init__(self,
                 z_size=10,         # Number of gird points for z
                 kappa_size=10,     # Number of grid points for kappa
                 eta_size=10,       # Number of grid points for eta
                 d_size=10,         # Number of grid points for d
                 kappa_min=0.0,     # Minimal grid point for kappa
                 kappa_max=2.0,     # Maximal grid point for kappa
                 d_min=0.0,         # Minimal grid point for d
                 d_max=10.0,        # Maximal grid point for d
                 rho_z=0.99,        # Autocorrelation coefficient of {log z_t}
                 del_z=np.sqrt(0.007),     # Standard deviation of {ϵ_t}
                 del_eta=np.sqrt(0.043),   # Standard deviation of {eta_t}
                 beta=0.99,         # Discount factor
                 sig=2.0,           # Risk aversion coefficient
                 gamma=0.355,       # Marginal rate of garnishment
                 r_bar=0.2):        # Debt interest rate
        
        self.z_size, self.kappa_size, self.eta_size, self.d_size = z_size, kappa_size, eta_size, d_size
        self.beta, self.sig, self.gamma, self.r_bar = beta, sig, gamma, r_bar
        self.rho_z, self.del_z, self.del_eta = rho_z, del_z, del_eta
        
        self.kappa_grid = np.linspace(kappa_min, kappa_max, kappa_size)  # Grid points for kappa
        self.kappa_prob = np.ones(kappa_size) / kappa_size               # Uniform distribution
        
        self.d_grid = np.linspace(d_min, d_max, d_size)  # Grid points for d
        
        mc = qe.tauchen(rho_z, del_z, n=z_size)    # Discretize {z_t} via Tauchen's method
        self.P = mc.P                              # The probability transition matrix of {z_t}
        self.z_grid = np.exp(mc.state_values)      # The discretized state values for {z_t}
        
        disc_eta = qe.tauchen(0., del_eta, n=eta_size)    # Discretize {η_t} via Tauchen's method
        self.eta_prob = disc_eta.P[0,:]                   # The probability density function of {η_t}
        self.eta_grid = np.exp(disc_eta.state_values)     # The discretized state values for {η_t}

In [3]:
def operator_factory(cb):
    # Simplify parameters
    d_grid, z_grid, eta_grid, kappa_grid = cb.d_grid, cb.z_grid, cb.eta_grid, cb.kappa_grid
    beta, sig, gamma, r_bar, P, eta_prob = cb.beta, cb.sig, cb.gamma, cb.r_bar, cb.P, cb.eta_prob
    z_size, eta_size, kappa_size, d_size = cb.z_size, cb.eta_size, cb.kappa_size, cb.d_size
    
    @njit
    def q(z):
        return 1 + 0.1 * z
    
    @njit
    def u(c):
        return (c**(1 - sig)) / (1 - sig)
    
    @njit
    def T(vR, vB, vE):
        """
        The standard Bellman oparator
        """
        vR_new = np.empty_like(vR)
        vB_new = np.empty_like(vB)
        vE_new = np.empty_like(vE)
        
        # First update vR
        # Here's all states
        for i_d, d in enumerate(d_grid):
            for i_z, z in enumerate(z_grid):
                for i_eta, η in enumerate(eta_grid):
                    for i_kappa, κ in enumerate(kappa_grid):
                        # For each state, eval RHS of Bellman at all dp and record largest
                        current_max = -1e10
                        for i_dp, dp in enumerate(d_grid):
                            # Compute the expectation
                            e = 0.0
                            for i_zp in range(z_size):
                                for i_etap in range(eta_size):
                                    for i_kappap in range(kappa_size):
                                        e += max(vR[i_dp, i_zp, i_etap, i_kappap], 
                                                 vB[i_zp, i_etap]) * P[i_z, i_zp] * eta_prob[i_etap]
                            e = e * (1 / kappa_size)
                            candidate = u(dp * q(z) - κ - d + η * z) + beta * e
                            if candidate > current_max:
                                current_max = candidate
                        # Largest recorded is new value
                        vR_new[i_d, i_z, i_eta, i_kappa] = current_max
                        
        # Next update vB
        # Here's all states
        for i_z, z in enumerate(z_grid):
            for i_eta, η in enumerate(eta_grid):
                # Compute the expectation
                e = 0.0
                for i_zp in range(z_size):
                    for i_etap in range(eta_size):
                        for i_kappap in range(kappa_size):
                            e += max(vR[0, i_zp, i_etap, i_kappap], 
                                     vE[i_zp, i_etap, i_kappap]) * P[i_z, i_zp] * eta_prob[i_etap]
                e = e * (1 / kappa_size)
                vB_new[i_z, i_eta] = ((1 - gamma) * z * η) + beta * e
        
        # Finally, update vE
        # Here's all the states
        for i_z, z in enumerate(z_grid):
            for i_eta, η in enumerate(eta_grid):
                for i_kappa, κ in enumerate(kappa_grid):
                    i_d_hat = np.searchsorted(d_grid, (κ - gamma * z * η) * (1 + r_bar))
                    # Compute the expectation
                    e = 0.0
                    for i_zp in range(z_size):
                        for i_etap in range(eta_size):
                            for i_kappap in range(kappa_size):
                                e += max(vR[i_d_hat, i_zp, i_etap, i_kappap], 
                                         vB[i_zp, i_etap]) * P[i_z, i_zp] * eta_prob[i_etap]
                    e = e * (1 / kappa_size)
                    vE_new[i_z, i_eta, i_kappa] = u((1 - gamma) * z * η) + beta * e
        
        return vR_new, vB_new, vE_new
   

    @njit
    def S(gD, gE):
        """
        The Refactored Bellman operator
        """
        gD_new = np.empty_like(gD)
        gE_new = np.empty_like(gE)
        
        # First update gD
        # Step through all states
        for i_z, z in enumerate(z_grid):
            for i_dp, dp in enumerate(d_grid):
                e = 0.0 # Will hold the expectation
                for i_zp, zp in enumerate(z_grid):
                    for i_etap, etap in enumerate(eta_grid):
                        for i_kappa, kappap in enumerate(kappa_grid):
                            # Compute the max of two terms, L and R (left and right)
                            # Start with R
                            c_tilde = (1 - gamma) * zp * etap
                            R = u(c_tilde) + beta * gE[i_zp]
                            # Next, L
                            current_max = -1e10
                            for i_dpp, dpp in enumerate(d_grid):
                                util = u(zp * etap + q(zp) * dpp - dp - kappap)
                                m = util + beta * gD[i_zp, i_dpp]
                                if m > current_max:
                                    current_max = m
                            L = current_max
                            e += max(L,R) * P[i_z, i_zp] * eta_prob[i_etap]
                e = e * (1 / kappa_size)
                gD_new[i_z, i_dp] = e
                
        # Next update gE
        # Step through all states:
        for i_z, z in enumerate(z_grid):
            e = 0.0 # Will hold the expectation
            for i_zp, zp in enumerate(z_grid):
                for i_etap, etap in enumerate(eta_grid):
                    for i_kappap, kappap in enumerate(kappa_grid):
                        # Compute the max of two terms, L and R (Left and Right)
                        # Start with R
                        c_tilde = (1 - gamma) * zp * etap
                        i_d_tilde = np.searchsorted(d_grid, (kappap - gamma * zp * etap) * (1 + r_bar))
                        R = u(c_tilde) + beta * gD[i_zp, i_d_tilde]
                        # Next, L
                        current_max = -1e10
                        for i_dpp, dpp in enumerate(d_grid):
                            util = u(zp * etap + q(zp) * dpp - kappap)
                            m = util + beta * gD[i_zp, i_dpp]
                            if m > current_max:
                                current_max = m
                        L = current_max
                        e += max(L,R) * P[i_z, i_zp] * eta_prob[i_etap]
            e = e * (1 / kappa_size)
            gE_new[i_z] = e
            
        return gD_new, gE_new
    
    return T, S

In [4]:
def solve_model(cb, method='vfi', tol=1e-4, iter_max=5000):
    """
    Solve the model via the selected method. 
    -----------------------------------------
    'vfi'  : value function iteration based on 
             the standard Bellman operator
    'rvfi' : refactored value function iteration based on 
             the refactored Bellman operator
    """
    z_size, eta_size, kappa_size, d_size = cb.z_size, cb.eta_size, cb.kappa_size, cb.d_size
    
    i = 0           # Index of iteration 
    eps = tol + 1.  # Initial error level
    
    if method == 'vfi':
        # Initial guess of vR, vB, vE
        vR = np.ones((d_size, z_size, eta_size, kappa_size))
        vB = np.ones((z_size, eta_size))
        vE = np.ones((z_size, eta_size, kappa_size))
        
        T = operator_factory(cb)[0]  # The standard Bellman operator
        
        while eps > tol and i < iter_max:
            vR_new, vB_new, vE_new = T(vR, vB, vE)
            eps_R = np.max(np.abs(vR_new - vR))
            eps_B = np.max(np.abs(vB_new - vB))
            eps_E = np.max(np.abs(vB_new - vB))
            eps = max(eps_R, eps_B, eps_E)
            vR, vB, vE = vR_new, vB_new, vE_new
            i += 1
        if i == iter_max:
            print("")
            print("Failed to converge!")
            print("")
        return i, eps, vR, vB, vE
    else:
        # Initial guess of gD, gE
        gD = np.ones((z_size, d_size))
        gE = np.ones(z_size)
        
        S = operator_factory(cb)[1]  # The refactored Bellman operator
        
        while eps > tol and i < iter_max:
            gD_new, gE_new = S(gD, gE)
            eps_D = np.max(np.abs(gD_new - gD))
            eps_E = np.max(np.abs(gE_new - gE))
            eps = max(eps_D, eps_E)
            gD, gE = gD_new, gE_new
            i += 1
        if i == iter_max:
            print("")
            print("Failed to converge!")
            print("")
        return i, eps, gD, gE

## Standard Value Function Iteration 

In [22]:
print("----------------------------------------")
print(" Vlue Function Iteration : Varying beta")
print("----------------------------------------")

beta_vals = [0.94, 0.95, 0.96, 0.97, 0.98]
size_vals = [10, 12, 14, 16, 18, 20]

time_vfi = np.empty((len(size_vals), len(beta_vals)))

for i_beta, beta in enumerate(beta_vals):
    for i_size, size in enumerate(size_vals):
        start_time = time.time()  # Start the clock
        cb = ConsumerBankruptcy(z_size=size, kappa_size=size, eta_size=size, 
                                d_size=size, beta=beta)
        i, eps, vR, vB, vE = solve_model(cb)  
        time_vfi[i_size, i_beta] = time.time() - start_time  # Calculate time taken

        print(" beta       : ", beta)
        print(" grid size  : ", size)
        print("")
        print(" time taken : ", time_vfi[i_size, i_beta])
        print("-------------------------------------")

----------------------------------------
 Vlue Function Iteration : Varying beta
----------------------------------------
 beta       :  0.94
 grid size  :  10

 time taken :  20.754160165786743
-------------------------------------
 beta       :  0.94
 grid size  :  12

 time taken :  92.78361487388611
-------------------------------------
 beta       :  0.94
 grid size  :  14

 time taken :  321.5344548225403
-------------------------------------
 beta       :  0.94
 grid size  :  16

 time taken :  1445.528903722763
-------------------------------------
 beta       :  0.94
 grid size  :  18

 time taken :  2412.8378913402557
-------------------------------------
 beta       :  0.94
 grid size  :  20

 time taken :  5591.374099493027
-------------------------------------
 beta       :  0.95
 grid size  :  10

 time taken :  24.166757345199585
-------------------------------------
 beta       :  0.95
 grid size  :  12

 time taken :  110.06483340263367
--------------------------------

In [23]:
# Note: In this group of experiments, we set the discount factor β=0.97.

print("----------------------------------------")
print(" Vlue Function Iteration : Varying {z_t}")
print("----------------------------------------")

rhoz_vals = [0.96, 0.97, 0.98, 0.995]
delz_vals = np.sqrt([0.01, 0.04])
size_vals = [10, 12, 14, 16]

time_vfi = np.empty((len(size_vals), len(rhoz_vals), len(delz_vals)))

for i_rhoz, rhoz in enumerate(rhoz_vals):
    for i_size, size in enumerate(size_vals):
        for i_delz, delz in enumerate(delz_vals):
            start_time = time.time()  # Start the clock
            cb = ConsumerBankruptcy(z_size=size, kappa_size=size, eta_size=size, d_size=size, 
                                    beta=0.97, rho_z=rhoz, del_z=delz)   
            i, eps, vR, vB, vE = solve_model(cb)  
            time_vfi[i_size, i_rhoz, i_delz] = time.time() - start_time  # Calculate time taken
            
            print(" rho_z      : ", rhoz)
            print(" del_z      : ", delz)
            print(" grid size  : ", size)
            print("")
            print(" time taken : ", time_vfi[i_size, i_rhoz, i_delz])
            print("-------------------------------------")

----------------------------------------
 Vlue Function Iteration : Varying {z_t}
----------------------------------------
 rho_z      :  0.96
 del_z      :  0.1
 grid size  :  10

 time taken :  40.83875846862793
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.2
 grid size  :  10

 time taken :  46.76874327659607
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.1
 grid size  :  12

 time taken :  171.09439754486084
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.2
 grid size  :  12

 time taken :  185.87988424301147
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.1
 grid size  :  14

 time taken :  586.1810786724091
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.2
 grid size  :  14

 time taken :  693.9127912521362
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.1
 grid size  :  16

 time taken :  1824.3992743492126
---------

In [24]:
# Note: In this group of experiments, we set the discount factor β=0.98.

print("----------------------------------------")
print(" Vlue Function Iteration : Varying {z_t}")
print("----------------------------------------")

rhoz_vals = [0.96, 0.97, 0.98, 0.995]
delz_vals = np.sqrt([0.01, 0.04])
size_vals = [10, 12, 14, 16]

time_vfi = np.empty((len(size_vals), len(rhoz_vals), len(delz_vals)))

for i_rhoz, rhoz in enumerate(rhoz_vals):
    for i_size, size in enumerate(size_vals):
        for i_delz, delz in enumerate(delz_vals):
            start_time = time.time()  # Start the clock
            cb = ConsumerBankruptcy(z_size=size, kappa_size=size, eta_size=size, d_size=size, 
                                    beta=0.98, rho_z=rhoz, del_z=delz)   
            i, eps, vR, vB, vE = solve_model(cb)  
            time_vfi[i_size, i_rhoz, i_delz] = time.time() - start_time  # Calculate time taken
            
            print(" rho_z      : ", rhoz)
            print(" del_z      : ", delz)
            print(" grid size  : ", size)
            print("")
            print(" time taken : ", time_vfi[i_size, i_rhoz, i_delz])
            print("-------------------------------------")

----------------------------------------
 Vlue Function Iteration : Varying {z_t}
----------------------------------------
 rho_z      :  0.96
 del_z      :  0.1
 grid size  :  10

 time taken :  60.8260498046875
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.2
 grid size  :  10

 time taken :  68.43030261993408
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.1
 grid size  :  12

 time taken :  255.11490941047668
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.2
 grid size  :  12

 time taken :  282.58530473709106
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.1
 grid size  :  14

 time taken :  881.8393161296844
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.2
 grid size  :  14

 time taken :  1045.875969171524
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.1
 grid size  :  16

 time taken :  2739.1039531230927
----------

In [5]:
# A single experiment

start_time_vfi = time.time() # Start the clock

cb = ConsumerBankruptcy(z_size=10, kappa_size=10, eta_size=10, d_size=10, beta=0.98)
i, eps, vR, vB, vE = solve_model(cb)

time_vfi = time.time() - start_time_vfi # Calculate time taken

print(f"Terminated at iteration {i} with error {eps}.")
print("")
print("Time taken VFI: ", time_vfi)
print("------------------------------------------------------")
print(" (z, kappa, eta, d) : ", (cb.z_size, cb.kappa_size, cb.eta_size, cb.d_size))
print(" beta    = ", cb.beta)
print(" rho_z   = ", cb.rho_z)
print(" del_z   = ", cb.del_z)
print(" del_eta = ", cb.del_eta)

Terminated at iteration 2984 with error 0.0.

Time taken VFI:  10355.201855182648
------------------------------------------------------
 (z, kappa, eta, d) :  (15, 15, 15, 15)
 beta    =  0.99
 rho_z   =  0.99
 del_z   =  0.08366600265340755
 del_eta =  0.2073644135332772


## Refactored Value Function Iteration

In [5]:
print("---------------------------------------------------")
print(" Refactored Vlue Function Iteration : Varying beta")
print("---------------------------------------------------")

beta_vals = [0.94, 0.95, 0.96, 0.97, 0.98]
size_vals = [10, 12, 14, 16, 18, 20]

time_rvfi = np.empty((len(size_vals), len(beta_vals)))

for i_beta, beta in enumerate(beta_vals):
    for i_size, size in enumerate(size_vals):
        start_time = time.time()  # Start the clock
        cb = ConsumerBankruptcy(z_size=size, kappa_size=size, eta_size=size, 
                                d_size=size, beta=beta)
        i, eps, gD, gE = solve_model(cb, method='rvfi')  
        time_rvfi[i_size, i_beta] = time.time() - start_time  # Calculate time taken

        print(" beta       : ", beta)
        print(" grid size  : ", size)
        print("")
        print(" time taken : ", time_rvfi[i_size, i_beta])
        print("-------------------------------------")

---------------------------------------------------
 Refactored Vlue Function Iteration : Varying beta
---------------------------------------------------
 beta       :  0.94
 grid size  :  10

 time taken :  1.2686107158660889
-------------------------------------
 beta       :  0.94
 grid size  :  12

 time taken :  1.4603912830352783
-------------------------------------
 beta       :  0.94
 grid size  :  14

 time taken :  2.547006368637085
-------------------------------------
 beta       :  0.94
 grid size  :  16

 time taken :  4.918048858642578
-------------------------------------
 beta       :  0.94
 grid size  :  18

 time taken :  10.939471960067749
-------------------------------------
 beta       :  0.94
 grid size  :  20

 time taken :  19.775496244430542
-------------------------------------
 beta       :  0.95
 grid size  :  10

 time taken :  0.8921825885772705
-------------------------------------
 beta       :  0.95
 grid size  :  12

 time taken :  1.55649590492248

In [20]:
# Note: In this group of experiments, we set the discount factor β=0.97.

print("----------------------------------------------------")
print(" Refactored Vlue Function Iteration : Varying {z_t}")
print("----------------------------------------------------")

rhoz_vals = [0.96, 0.97, 0.98, 0.995]
delz_vals = np.sqrt([0.01, 0.04])
size_vals = [10, 12, 14, 16]

time_rvfi = np.empty((len(size_vals), len(rhoz_vals), len(delz_vals)))

for i_rhoz, rhoz in enumerate(rhoz_vals):
    for i_size, size in enumerate(size_vals):
        for i_delz, delz in enumerate(delz_vals):
            start_time = time.time()  # Start the clock
            cb = ConsumerBankruptcy(z_size=size, kappa_size=size, eta_size=size, d_size=size, 
                                    beta=0.97, rho_z=rhoz, del_z=delz)    # Note: discount factor β=0.97
            i, eps, gD, gE = solve_model(cb, method='rvfi')  
            time_rvfi[i_size, i_rhoz, i_delz] = time.time() - start_time  # Calculate time taken
            
            print(" rho_z      : ", rhoz)
            print(" del_z      : ", delz)
            print(" grid size  : ", size)
            print("")
            print(" time taken : ", time_rvfi[i_size, i_rhoz, i_delz])
            print("-------------------------------------")

----------------------------------------------------
 Refactored Vlue Function Iteration : Varying {z_t}
----------------------------------------------------
 rho_z      :  0.96
 del_z      :  0.1
 grid size  :  10

 time taken :  1.232114553451538
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.2
 grid size  :  10

 time taken :  1.266636610031128
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.1
 grid size  :  12

 time taken :  2.0509798526763916
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.2
 grid size  :  12

 time taken :  2.2049012184143066
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.1
 grid size  :  14

 time taken :  4.162678480148315
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.2
 grid size  :  14

 time taken :  4.717677116394043
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.1
 grid size  :  16

 time ta

In [25]:
# Note: In this group of experiments, we set the discount factor β=0.98.

print("----------------------------------------------------")
print(" Refactored Vlue Function Iteration : Varying {z_t}")
print("----------------------------------------------------")

rhoz_vals = [0.96, 0.97, 0.98, 0.995]
delz_vals = np.sqrt([0.01, 0.04])
size_vals = [10, 12, 14, 16]

time_rvfi = np.empty((len(size_vals), len(rhoz_vals), len(delz_vals)))

for i_rhoz, rhoz in enumerate(rhoz_vals):
    for i_size, size in enumerate(size_vals):
        for i_delz, delz in enumerate(delz_vals):
            start_time = time.time()  # Start the clock
            cb = ConsumerBankruptcy(z_size=size, kappa_size=size, eta_size=size, d_size=size, 
                                    beta=0.98, rho_z=rhoz, del_z=delz)    
            i, eps, gD, gE = solve_model(cb, method='rvfi')  
            time_rvfi[i_size, i_rhoz, i_delz] = time.time() - start_time  # Calculate time taken
            
            print(" rho_z      : ", rhoz)
            print(" del_z      : ", delz)
            print(" grid size  : ", size)
            print("")
            print(" time taken : ", time_rvfi[i_size, i_rhoz, i_delz])
            print("-------------------------------------")

----------------------------------------------------
 Refactored Vlue Function Iteration : Varying {z_t}
----------------------------------------------------
 rho_z      :  0.96
 del_z      :  0.1
 grid size  :  10

 time taken :  1.4865021705627441
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.2
 grid size  :  10

 time taken :  1.6470115184783936
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.1
 grid size  :  12

 time taken :  2.981755495071411
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.2
 grid size  :  12

 time taken :  3.4616827964782715
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.1
 grid size  :  14

 time taken :  6.249145269393921
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.2
 grid size  :  14

 time taken :  7.470057487487793
-------------------------------------
 rho_z      :  0.96
 del_z      :  0.1
 grid size  :  16

 time t

In [18]:
# A single experiment

start_time_rvfi = time.time() # Start the clock

cb = ConsumerBankruptcy(z_size=10, kappa_size=10, eta_size=10, d_size=10, beta=0.94)
i, eps, gD, gE = solve_model(cb, method='rvfi')

time_rvfi = time.time() - start_time_rvfi # Calculate time taken

print(f"Terminated at iteration {i} with error {eps}.")
print("")
print("Time taken RVFI: ", time_rvfi)
print("------------------------------------------------------")
print(" (z, kappa, eta, d) : ", (cb.z_size, cb.kappa_size, cb.eta_size, cb.d_size))
print(" beta    = ", cb.beta)
print(" rho_z   = ", cb.rho_z)
print(" del_z   = ", cb.del_z)
print(" del_eta = ", cb.del_eta)

Terminated at iteration 189 with error 9.51158017699072e-05.

Time taken RVFI:  0.8792188167572021
------------------------------------------------------
 (z, kappa, eta, d) :  (10, 10, 10, 10)
 beta    =  0.94
 rho_z   =  0.99
 del_z   =  0.08366600265340755
 del_eta =  0.2073644135332772
